In [22]:

import yfinance as yf
import pandas as pd
import pandas_ta as ta
from nsepython import *
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [23]:
import requests
import pandas as pd

# URL of the Nifty 500 companies list
url = 'https://nsearchives.nseindia.com/content/indices/ind_nifty500list.csv'

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Check if request was successful

    # Save CSV content to a local file
    with open('ind_nifty500list.csv', 'wb') as file:
        file.write(response.content)

    # Load CSV file into a DataFrame
    nifty500_data = pd.read_csv('ind_nifty500list.csv')
    nifty500_tickers = [f"{symbol}.NS" for symbol in nifty500_data['Symbol']]
    print("Downloaded Nifty 500 tickers successfully!")

except Exception as e:
    print("Error downloading or reading Nifty 500 symbols:", e)


Downloaded Nifty 500 tickers successfully!


In [24]:
nifty500_data.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026


In [25]:
nifty500_tickers[:5]

['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'AIAENG.NS']

In [26]:
# Parameters for RSI
rsi_period = 14
rsi_threshold = 40
sma_length = 14


In [27]:
# Function to calculate RSI and check if it's below the threshold
def get_low_rsi_stocks(tickers, rsi_period, rsi_threshold, sma_length):
    low_rsi_stocks = []
    
    for ticker in tickers:
        try:
            # Fetch weekly data for each stock
            stock_data = yf.download(ticker, period="1y", interval="1wk")
            
            # Ensure we have enough data
            if len(stock_data) < rsi_period:
                continue

            # Calculate RSI with pandas_ta
            stock_data['RSI'] = ta.rsi(stock_data['Close'], length=rsi_period)
            
            # Calculate SMA of the RSI
            stock_data['RSI_SMA'] = stock_data['RSI'].rolling(window=sma_length).mean()
            
            # Check if the last RSI value is below the threshold
            if stock_data['RSI'].iloc[-1] < rsi_threshold:
                low_rsi_stocks.append(ticker)
        
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
    
    return low_rsi_stocks

In [28]:
# Get list of low RSI stocks
low_rsi_stocks = get_low_rsi_stocks(nifty500_tickers, rsi_period, rsi_threshold, sma_length)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [29]:
# Output the stocks with RSI < 40
print("Nifty 500 Companies with RSI below 40:", low_rsi_stocks)

Nifty 500 Companies with RSI below 40: ['AARTIIND.NS', 'ATGL.NS', 'BANKINDIA.NS', 'BIRLACORPN.NS', 'BSOFT.NS', 'CGCL.NS', 'CONCOR.NS', 'CREDITACC.NS', 'DATAPATTNS.NS', 'EASEMYTRIP.NS', 'EQUITASBNK.NS', 'MEDANTA.NS', 'GPIL.NS', 'GAEL.NS', 'GMDCLTD.NS', 'IDFCFIRSTB.NS', 'IRCON.NS', 'ITI.NS', 'IOB.NS', 'IRCTC.NS', 'INTELLECT.NS', 'J&KBANK.NS', 'LEMONTREE.NS', 'MAHSEAMLES.NS', 'MAHLIFE.NS', 'MRPL.NS', 'PNB.NS', 'RBLBANK.NS', 'RAYMOND.NS', 'SWSOLAR.NS', 'SPARC.NS', 'TRIDENT.NS', 'UCOBANK.NS', 'UJJIVANSFB.NS', 'UNIONBANK.NS', 'IDEA.NS', 'ZEEL.NS']


In [14]:
# Function to plot candlestick and RSI
def plot_candlestick_with_rsi(ticker, rsi_period=14):
    # Download stock data for the last year in weekly intervals
    stock_data = yf.download(ticker, period="1y", interval="1wk")
    
    # Calculate RSI
    stock_data['RSI'] = ta.rsi(stock_data['Close'], length=rsi_period)
    
    # Create a subplot with shared x-axis
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.1, subplot_titles=(f"{ticker} Candlestick Chart", "RSI"))

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=stock_data.index,
        open=stock_data['Open'],
        high=stock_data['High'],
        low=stock_data['Low'],
        close=stock_data['Close'],
        name="Candlestick"
    ), row=1, col=1)

    # RSI line
    fig.add_trace(go.Scatter(
        x=stock_data.index, y=stock_data['RSI'],
        mode='lines',
        name='RSI'
    ), row=2, col=1)
    
    # Add RSI threshold lines
    fig.add_hline(y=30, line=dict(color="green", dash="dash"), row=2, col=1)
    fig.add_hline(y=70, line=dict(color="red", dash="dash"), row=2, col=1)
    
    # Update layout
    fig.update_layout(title=f"{ticker} Candlestick with RSI", xaxis_title="Date",
                      yaxis_title="Price", height=800)
    fig.update_yaxes(title_text="RSI", row=2, col=1)

    # Show the figure
    fig.show()




In [15]:
# Plot each stock in the low RSI stocks list
for ticker in low_rsi_stocks:
    plot_candlestick_with_rsi(ticker, rsi_period=14)

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


In [18]:
import yfinance as yf
import pandas_ta as ta
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Parameters
rsi_period = 14
rsi_threshold = 40

# Interactive multi-stock plot function
def plot_multi_stock_with_rsi(tickers, rsi_period):
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=("Candlestick Chart", "RSI")
    )

    # For each ticker, add a trace for OHLC and RSI, and add it to the figure
    for i, ticker in enumerate(tickers):
        # Fetch data
        stock_data = yf.download(ticker, period="1y", interval="1wk")
        stock_data['RSI'] = ta.rsi(stock_data['Close'], length=rsi_period)

        # Candlestick trace
        fig.add_trace(
            go.Candlestick(
                x=stock_data.index,
                open=stock_data['Open'],
                high=stock_data['High'],
                low=stock_data['Low'],
                close=stock_data['Close'],
                name=f"{ticker} Candlestick",
                visible=(i == 0)  # Show only the first stock initially
            ),
            row=1, col=1
        )

        # RSI trace
        fig.add_trace(
            go.Scatter(
                x=stock_data.index,
                y=stock_data['RSI'],
                mode="lines",
                name=f"{ticker} RSI",
                visible=(i == 0)
            ),
            row=2, col=1
        )

    # Configure slider steps to toggle between stocks
    steps = []
    for i, ticker in enumerate(tickers):
        step = dict(
            method="update",
            args=[
                {"visible": [False] * (2 * len(tickers))},
                {"title": f"{ticker} Candlestick and RSI"}
            ],
        )
        step["args"][0]["visible"][2 * i] = True  # Show candlestick of ticker
        step["args"][0]["visible"][2 * i + 1] = True  # Show RSI of ticker
        steps.append(step)

    # Create slider
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Stock: "},
        pad={"t": 50},
        steps=steps
    )]

    # Update figure layout
    fig.update_layout(
        sliders=sliders,
        title=f"Candlestick and RSI for Selected Nifty 500 Stocks",
        xaxis_title="Date",
        yaxis_title="Price",
        yaxis2_title="RSI",
        height=800
    )
    fig.update_yaxes(title_text="RSI", row=2, col=1)
    fig.update_layout(template="plotly_dark", showlegend=False)

    # Display the plot in full-screen mode
    fig.show(config={'displayModeBar': True, 'scrollZoom': True})

# Call the function with the list of low RSI stocks
plot_multi_stock_with_rsi(low_rsi_stocks[:10], rsi_period)  # Using the first 10 stocks for demo


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [19]:
import plotly.io as pio

# Use 'browser' to open the figure in a new browser window/tab
pio.renderers.default = 'browser'

# Your existing plotting code here
plot_multi_stock_with_rsi(low_rsi_stocks[:10], rsi_period)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


ImportError: cannot import name 'QtWebEngineWidgets' from 'PyQt5' (c:\Users\ASUS\Miniconda3\lib\site-packages\PyQt5\__init__.py)